# Worksheet 13

Name: Oliver Samuels<br>
UID: U45944770<br>

### Topics

- Support Vector Machines
- Recommender Systems

## Support Vector Machines

Follow along in class to implement the perceptron algorithm and create an animation of the algorithm.

In [1]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

TEMPFILE = "temp.png"
CENTERS = [[0, 1], [1, 0]]

# Dataset
X, labels = datasets.make_blobs(n_samples=10, centers=CENTERS, cluster_std=0.2, random_state=0)
Y = np.array(list(map(lambda x : -1 if x == 0 else 1, labels.tolist())))

# Initializing w and b
w = np.array([1, 1])
b = 0

# Perceptron Parameters
epochs = 100
alpha = .05
expanding_rate = .99
retracting_rate = 1.1

def snap(x, w, b, error):
    """
        Plot the street induced by w and b.
        Circle the point x in red if it was
        misclassified or in yellow if it was
        classified correctly.
    """

    xplot = np.linspace(-3, 3)
    cs = np.array([x for x in 'gb'])

    svm = -(w[0]/w[1]* xplot -b / w[1])
    left_svm = -(w[0]/w[1]*xplot -b / w[1] - 1/w[1])
    right_svm = -(w[0]/w[1]*xplot -b / w[1] - 1/w[1])

    fig, ax = plt.subplots()
    ax.scatter(X[:,0],X[:,1],color=cs[labels].tolist(), s=50, alpha=0.8)
    if error:
        ax.add_patch(plt.Circle((x[0], x[1]), .2, color='r',fill=False))
    else:
        ax.add_patch(plt.Circle((x[0], x[1]), .2, color='y',fill=False))
    ax.plot(xplot, left_svm, 'r--', lw=2)
    ax.plot(xplot, svm, 'r-', lw=2)
    ax.plot(xplot, right_svm, 'r--', lw=2)
    ax.set_xlim(min(X[:, 0]) - 1, max(X[:,0]) + 1)
    ax.set_ylim(min(X[:, 1]) - 1, max(X[:,1]) + 1)
    fig.savefig(TEMPFILE)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE)))
def accuracy(w,b):
    acc = 0
    for x in X:
        ypred = np.dot(w,x)+b
        if(ypred >= 0 and y>0) or(ypred < 0 and y<0):
            acc+=1
        return acc/len(X)


images = []
for _ in range(epochs):
    # pick a point from X at random
    i = np.random.randint(0,len(X))
    x, y = X[i], Y[i]
    error = False
    ypred = np.dot(w,x)+b
    images.append(snap(x, w, b, error))
    if(ypred >= 0 and y>0) or(ypred < 0 and y<0):
        #correctly classified
        if(ypred <= 1 and ypred >= -1):
            w = w+alpha * y * x *retracting_rate
            b += alpha * y * retracting_rate
        else:
            w = w*expanding_rate
            b *= expanding_rate
    else:
        #missclassified
        w =  w+ alpha * y * x * expanding_rate
        b += alpha * y * expanding_rate

    print("training accuracy = ", accuracy(w,b))



images[0].save(
    'svm.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=100
)

training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.1
training accuracy =  0.1
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.1
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.1
training accuracy =  0.1
training accuracy =  0.1
training accuracy =  0.1
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.1
training accuracy =  0.1
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.1
training accuracy =  0.1
training accuracy =  0.1
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.1
training accuracy =  0.1
training accuracy =  0.1
training accuracy =  0.0
training accuracy =  0.1
training accuracy =  0.0
training accuracy =  0.1
training accuracy =  0.0
training accuracy =  0.0
training accuracy =  0.1
training accuracy =  0.1


As we saw in class, the form
$$w^T x + b = 0$$
while simple, does not expose the inner product `<x_i, x_j>` which we know `w` depends on, having done the math. This is critical to applying the "kernel trick" which allows for learning non-linear decision boundaries. Let's modify the above algorithm to use the form
$$\sum_i \alpha_i <x_i, x> + b = 0$$

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

TEMPFILE = "temp.png"
CENTERS = [[0, 1], [1, 0]]

epochs = 100
learning_rate = .05
expanding_rate = .99
retracting_rate = 1.1

X, labels = datasets.make_blobs(n_samples=10, centers=CENTERS, cluster_std=0.2, random_state=0)
Y = np.array(list(map(lambda x : -1 if x == 0 else 1, labels.tolist())))

alpha_i = np.zeros((len(X),))
b = 0

def snap(x, alpha_i, b, error):
    # create a mesh to plot in
    h = .01  # step size in the mesh
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    meshData = np.c_[xx.ravel(), yy.ravel()]
    cs = np.array([x for x in 'gb'])
    fig, ax = plt.subplots()
    ax.scatter(X[:,0],X[:,1],color=cs[labels].tolist(), s=50, alpha=0.8)

    if error:
        ax.add_patch(plt.Circle((x[0], x[1]), .12, color='r',fill=False))
    else:
        ax.add_patch(plt.Circle((x[0], x[1]), .12, color='y',fill=False))
   
    Z = predict_many(alpha_i, b, meshData)
    Z = np.array([0 if z <=0 else 1 for z in Z]).reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=.5, cmap=plt.cm.Paired)
    fig.savefig(TEMPFILE)
    plt.close()
    return im.fromarray(np.asarray(im.open(TEMPFILE)))

def predict_many(alpha_i, b, Z):
    res = []
    for i in range(len(Z)):
        res.append(predict(alpha_i, b, Z[i]))
    return np.array(res)

def predict(alpha_i, b, x):
    wx = 0
    for i in range (len(X)):
        wx += alpha_i[i]*np.dot(X[i],x)
    return wx+b

images = []
for _ in range(epochs):
    # pick a point from X at random
    i = np.random.randint(0, len(X))
    error = False
    x, y = X[i], Y[i]
    
    ypred = predict(alpha_i,b,x)
    images.append(snap(x, w, b, error))
    if(ypred >= 0 and y>0) or(ypred < 0 and y<0):
        #correctly classified
        if(ypred <= 1 and ypred >= -1):
            alpha_i[i] += learning_rate*y
            b += alpha * y * retracting_rate
            alpha_i = alpha_i*retracting_rate
        else:
            alpha_i = alpha_i*expanding_rate
            b *= expanding_rate
    else:
        #missclassified
            alpha_i[i] += learning_rate*y
            b += alpha * y * expanding_rate
            alpha_i = alpha_i*expanding_rate
    # print("training accuracy = ", accuracy(w,b))
    images.append(snap(x, alpha_i, b, error))

images[0].save(
    'svm_dual.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=100
)

Write a configurable kernel function to apply in lieu of the dot product. Try it out on a dataset that is not linearly separable.

In [ ]:
def polynomial(x_i, x_j, c, n):
    return (np.dot(x_i, x_j) + c) ** n

## Recommender Systems

In the example in class of recommending movies to users we used the movie rating as a measure of similarity between users and movies and thus the predicted rating for a user is a proxy for how highly a movie should be recommended. So the higher the predicted rating for a user, the higher a recommendation it would be.

a) Consider a streaming platform that only has "like" or "dislike" (no 1-5 rating). Describe how you would build a recommender system in this case.

b) Describe 3 challenges of building a recommender system

c) Why is SVD not an option for collaborative filtering?